In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#torch, nn, optim for building and training neural networks


In [ ]:
def make_batch():
    input_batch, target_batch = [], []

    for seq in seq_data:#each word in seq_data is slit
        input = [word_dict[n] for n in seq[:-1]]
        target = word_dict[seq[-1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [ ]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)
        self.W = nn.Linear(n_hidden, n_class, bias=False)
        self.b = nn.Parameter(torch.ones([n_class]))

    def forward(self, X):
        input = X.transpose(0, 1)  # Shape: [n_step, batch_size, n_class]
        hidden_state = torch.zeros(1, len(X), n_hidden)
        cell_state = torch.zeros(1, len(X), n_hidden)

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  # Take the output of the last LSTM cell
        model = self.W(outputs) + self.b
        return model

In [ ]:
if __name__ == '__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
    word_dict = {n: i for i, n in enumerate(char_arr)}
    number_dict = {i: w for i, w in enumerate(char_arr)}
    n_class = len(word_dict)

    seq_data = [ 'planet', 'rocket', 'follow', 'magnet', 'candle', 'rivers']

In [ ]:
model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 0100 cost = 0.147832
Epoch: 0200 cost = 0.004510
Epoch: 0300 cost = 0.002115
Epoch: 0400 cost = 0.001218
Epoch: 0500 cost = 0.000828
Epoch: 0600 cost = 0.000609
Epoch: 0700 cost = 0.000470
Epoch: 0800 cost = 0.000376
Epoch: 0900 cost = 0.000308
Epoch: 1000 cost = 0.000257


In [ ]:
inputs = [sen[:5] for sen in seq_data]

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

['plane', 'rocke', 'follo', 'magne', 'candl', 'river'] -> ['t', 't', 'w', 't', 'e', 's']
